In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(H5weaver)
library(hise)
library(tidyverse)
library(SeuratObject)
library(ggrepel)
library(SeuratDisk)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: Matrix

Loading required package: rhdf5


Attaching package: ‘H5weaver’


The following objects are masked from ‘package:rhdf5’:

    h5dump, h5ls


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.3     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ 

## Creating Seurat Objects from h5 outs from Cellranger

### Reading h5 files into memory

In [2]:
h5s <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/05_Data/scrublet_fig5', 
    pattern = 'filtered_feature_bc_matrix.h5$',
    full.names = TRUE, 
    recursive = TRUE
)


In [5]:
h5s[1]

[1] "/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/05_Data/scrublet_fig5/BL05012_CD3_CD28_sample_filtered_feature_bc_matrix.h5"

In [10]:
test <- strsplit(h5s[1], '/' )[[1]][9]
test <- strsplit(test, '_sample' )[[1]][1]
test

[1] "BL05012_CD3_CD28"

### Creating Seurat Objects

In [11]:
fully <- lapply(h5s, function(x){
    pro <- strsplit(x, '/' )[[1]][9]
    pro <- strsplit(pro, '_sample' )[[1]][1]
    mtx <- Read10X_h5(x) 
    so <- CreateSeuratObject(mtx,project=pro) 
    return(so) 
    })


In [12]:
head(fully[[1]])

,orig.ident,nCount_RNA,nFeature_RNA
,<fct>,<dbl>,<int>
AAACAAGCATAACCGAAACGGGAA-1,BL05012_CD3_CD28,2913,1592
AAACCAATCACGTTCAAACGGGAA-1,BL05012_CD3_CD28,6994,3513
AAACCAATCACTGTCAAACGGGAA-1,BL05012_CD3_CD28,12229,3982
AAACCAATCCACAATTAACGGGAA-1,BL05012_CD3_CD28,6612,2734
AAACCAATCCTAAATCAACGGGAA-1,BL05012_CD3_CD28,7654,2976
AAACCAATCGAACCGAAACGGGAA-1,BL05012_CD3_CD28,6523,2762
AAACCAATCGCTTGACAACGGGAA-1,BL05012_CD3_CD28,10994,3487
AAACCAATCTGCAACCAACGGGAA-1,BL05012_CD3_CD28,9766,3403
AAACCAGGTAAGAATCAACGGGAA-1,BL05012_CD3_CD28,5153,2347


In [13]:
scrub <- list.files(
    # Place your analysis in a folder by number experiment, or whatever might work for you
    # It helps if this worksheet in in the same folder
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/05_Data/scrublet_fig5',
    pattern = '.csv$', # change this to 'sample_raw_feature_bc_matrix.h5$' if you need the raw h5s
    full.names = TRUE, 
    recursive = TRUE
)


In [14]:
scrubs <- lapply(scrub,read.csv)

scrubs <- scrubs %>% lapply(function(x){
    rownames(x) <- x$X
    x$X <- NULL
    x$Barcodes <- rownames(x)
    return(x)
})

In [15]:
for (i in c(1:length(fully))){
    
    fully[[i]][[]]$Barcodes <- rownames(fully[[i]][[]])
    
    fully[[i]][[]] <- left_join(fully[[i]][[]], scrubs[[i]], by = 'Barcodes') # , by='row.names', all=TRUE
}

In [16]:
fully <- Reduce(merge,fully) %>% JoinLayers()


In [17]:
fully[["percent.mt"]] <- PercentageFeatureSet(fully, pattern = "^MT-")
fully <- subset(fully, subset = percent.mt < 5)


### Normalizing, running PCA and UMAP clustering

In [18]:
fully <- NormalizeData(fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)


Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  CD69, ICOS, SLA, MYC, PGGHG, GPR171, STAT5A, PKIA, NDFIP2, PGAP1 
	   MAL, TSHZ2, GFI1, WDR74, LAMP3, IRF4, SACS, CTPS1, BZW2, ITGA6 
	   MT2A, NUCB2, IFIT1, CD40LG, TFRC, RRP9, USP18, CISH, ODC1, DEAF1 
Negative:  ANPEP, SPI1, LYZ, CD300E, LILRB2, SEMA6B, KYNU, IFI30, NCF2, FCAR 
	   HCK, C5AR1, CST3, TLR4, ENG, S100A9, MPEG1, EMILIN2, SLC37A2, TLR2 
	   MS4A7, PLXNB2, CYBB, SERPINA1, PID1, TNFAIP2, ZNF385A, SLC7A11, VCAN, SRC 
PC_ 2 
Positive:  AOAH, SPN, CST7, NKG7, SLA2, CD300A, SYNE1, C1orf21, OASL, GNLY 
	   PRF1, KLRD1, PRR5L, HIST1H1C, FGFBP2, AUTS2, TRGC2, S1PR5, APOL3, GZMH 
	   CTSW, MYO1F, CCL5, ADGRG1, ZBP1, TRGC1, MYBL1, NCR1, TNFSF10, GK5 
Negative:  BCL2A1, FOSL1, KDM6B, SLC43A3, NME1, SLC7A1, PIM3, IRF8, EPOP, MYC 
	   TFRC, MFSD2A, LRRC8B, SLC7A5, IRF4, NFKBID, FABP5, SLAMF1, RRP9, SLC1A5 
	   BZW2, INSIG1, ODC1, TRAF4, SLC29A1, SLC16A1, PUS7, NA

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 127091
Number of edges: 4008535

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9479
Number of communities: 31
Elapsed time: 94 seconds


## Plotting

In [19]:
fully$tech <- substr(fully$orig.ident,21,44)
fully$donor <- substr(fully$orig.ident,13,19)


In [20]:
fully$tech <- substr(fully$orig.ident,9,25)
fully$donor <- substr(fully$orig.ident,1,7)


In [21]:
head(fully)

,orig.ident,nCount_RNA,nFeature_RNA,Barcodes,Doublet_Score,Predicted_Doublet,percent.mt,RNA_snn_res.0.5,seurat_clusters,tech,donor
,<chr>,<dbl>,<int>,<chr>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<chr>,<chr>
AAACAAGCATAACCGAAACGGGAA-1,BL05012_CD3_CD28,2913,1592,AAACAAGCATAACCGAAACGGGAA-1,0.05333333,False,0.00000000,21,21,CD3_CD28,BL05012
AAACCAATCACGTTCAAACGGGAA-1,BL05012_CD3_CD28,6994,3513,AAACCAATCACGTTCAAACGGGAA-1,0.01768570,False,0.48613097,18,18,CD3_CD28,BL05012
AAACCAATCACTGTCAAACGGGAA-1,BL05012_CD3_CD28,12229,3982,AAACCAATCACTGTCAAACGGGAA-1,0.13307985,False,1.30018808,14,14,CD3_CD28,BL05012
AAACCAATCCACAATTAACGGGAA-1,BL05012_CD3_CD28,6612,2734,AAACCAATCCACAATTAACGGGAA-1,0.03861004,False,0.00000000,12,12,CD3_CD28,BL05012
AAACCAATCCTAAATCAACGGGAA-1,BL05012_CD3_CD28,7654,2976,AAACCAATCCTAAATCAACGGGAA-1,0.02900232,False,0.06532532,12,12,CD3_CD28,BL05012
AAACCAATCGAACCGAAACGGGAA-1,BL05012_CD3_CD28,6523,2762,AAACCAATCGAACCGAAACGGGAA-1,0.18578352,False,0.13797333,9,9,CD3_CD28,BL05012
AAACCAATCGCTTGACAACGGGAA-1,BL05012_CD3_CD28,10994,3487,AAACCAATCGCTTGACAACGGGAA-1,0.08860759,False,0.08186283,10,10,CD3_CD28,BL05012
AAACCAATCTGCAACCAACGGGAA-1,BL05012_CD3_CD28,9766,3403,AAACCAATCTGCAACCAACGGGAA-1,0.07733813,False,0.05119803,9,9,CD3_CD28,BL05012
AAACCAGGTAAGAATCAACGGGAA-1,BL05012_CD3_CD28,5153,2347,AAACCAGGTAAGAATCAACGGGAA-1,0.02987698,False,0.00000000,12,12,CD3_CD28,BL05012


## Celltype label transfer

In [22]:
ref <- LoadH5Seurat(file = '/home/jupyter/pbmc_multimodal.h5seurat')


Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [23]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = fully,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
)


Normalizing query using reference SCT model

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 5404 anchors



In [24]:
fully <- TransferData(
    anchorset = anchors, 
    reference = ref, 
    query = fully,
    refdata = list(
        celltype.l1 = "celltype.l1",
        celltype.l2 = "celltype.l2",
        celltype.l3 = "celltype.l3",
        predicted_ADT = 'ADT'
    )
)

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Layer counts isn't present in the assay object; returning NULL”
Transfering 228 features onto reference data

Warning message:
“Layer counts isn't present in the assay object; returning NULL”


In [31]:
fully

An object of class Seurat 
18407 features across 127091 samples within 5 assays 
Active assay: RNA (18082 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 4 other assays present: prediction.score.celltype.l1, prediction.score.celltype.l2, prediction.score.celltype.l3, predicted_ADT
 2 dimensional reductions calculated: pca, umap

In [25]:
fully <- JoinLayers(fully)

In [26]:
fully$donor_tech <- paste0(fully$donor,'_',fully$tech)

In [27]:
saveRDS(fully, '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/05_Data/Fig_5_Final.rds')

In [28]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] SeuratDisk_0.0.0.9020 ggrepel_0.9.5         lubridate_1.9.3      
 [4] forcats_1.0.0         stringr_1.5.1         purrr_1.0.2          
 [7] readr_2.1.5           tidyr_1.3.1           tibble_3.2.1         
[10] ggplot2_3.4.3         tidyverse_2.0.0       hise_2.15.0          
[13] H5weaver_1.2.0        r

In [21]:
split <- SplitObject(fully, split.by='donor')


In [22]:
split

$CD28_sa
An object of class Seurat 
18407 features across 23930 samples within 5 assays 
Active assay: RNA (18082 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 4 other assays present: prediction.score.celltype.l1, prediction.score.celltype.l2, prediction.score.celltype.l3, predicted_ADT
 2 dimensional reductions calculated: pca, umap

$tim_sam
An object of class Seurat 
18407 features across 46697 samples within 5 assays 
Active assay: RNA (18082 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 4 other assays present: prediction.score.celltype.l1, prediction.score.celltype.l2, prediction.score.celltype.l3, predicted_ADT
 2 dimensional reductions calculated: pca, umap

$I_sampl
An object of class Seurat 
18407 features across 13961 samples within 5 assays 
Active assay: RNA (18082 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 4 other assays present: prediction.score.celltype.l1, prediction.sc

In [23]:
BL05012_cl <- split_cl$BL05012
BL05111_cl <- split_cl$BL05111
BL05731_cl <- split_cl$BL05731
PB02183_cl <- split_cl$PB02183


ERROR: Error in eval(expr, envir, enclos): object 'split_cl' not found


In [ ]:
BL05012_cl <- SplitObject(BL05012_cl, split.by='predicted.celltype.l1')
BL05111_cl <- SplitObject(BL05111_cl, split.by='predicted.celltype.l1')
BL05731_cl <- SplitObject(BL05731_cl, split.by='predicted.celltype.l1')
PB02183_cl <- SplitObject(PB02183_cl, split.by='predicted.celltype.l1')


In [ ]:
sessionInfo()